# NOAA-NCEI Data Scraping
---
Menne, Matthew J., Imke Durre, Bryant Korzeniewski, Shelley McNeill, Kristy Thomas, Xungang Yin, Steven Anthony, Ron Ray, Russell S. Vose, Byron E.Gleason, and Tamara G. Houston (2012): Global Historical Climatology Network - Daily (GHCN-Daily), Version 3. FIPS:06 PRCP. NOAA National Climatic Data Center. doi:10.7289/V5D21VHZ.

 Matthew J. Menne, Imke Durre, Russell S. Vose, Byron E. Gleason, and Tamara G. Houston, 2012: An Overview of the Global Historical Climatology Network-Daily Database. J. Atmos. Oceanic Technol., 29, 897-910. doi:10.1175/JTECH-D-11-00103.1.

https://www.ncdc.noaa.gov/cdo-web/webservices/v2#da
https://www.ncei.noaa.gov/pub/data/cdo/documentation/GHCND_documentation.pdf
https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt
ta

In [5]:
# Dependencies
import pandas as pd
import requests
import time
from api_key import noaa_key

## 1 Set Variables and Parameters
---

In [6]:
# Setup to retrive daily summary data from NOAA NCEI by year
year = 2012
export = False
dts = pd.date_range(str(year)+"-01-01", str(year)+"-12-31", freq="D")

In [ ]:
# Setup parameters for queries
url = "https://www.ncei.noaa.gov/cdo-web/api/v2/data?"
ds_id='GHCND'
dt_id='PRCP'
loc_id='FIPS:06' # California
limit=1000

## 2 Request API and Export Data
---

### 2.1 Initial Loop to Retrieve Data

In [ ]:
# Create List to catch failed requests
dts_chk = []
# loop through dates and retrieve data and write to csv by date
for dt in dts:
    d = str(dt.date())
    q_url = f"{url}datasetid={ds_id}&datatypeid={dt_id}&locationid={loc_id}&startdate={d}&enddate={d}&units=metric&limit={limit}"
    try: 
        response = requests.get(q_url, headers={'token': noaa_key}).json()
        df = pd.DataFrame(response['results'])
        df['station'] = df['station'].str.slice(6,17)
        df = df[['station','value']].rename(columns={'value':d})
        if export: df.to_csv(f'../outputs/ncei-noaa/{year}/csv/{d}.csv',index=False)
    except:
        dts_chk.append(d)
        print(f'{d}')
    time.sleep(1)
print(f'failed:{len(dts_chk)}')

### 2.2 Loop for Failed Dates

In [ ]:
# loop through failed dates and retrieve data and write to csv by date
for d in dts_chk:
    print(d)
    q_url = f"{url}datasetid={ds_id}&datatypeid={dt_id}&locationid={loc_id}&startdate={d}&enddate={d}&units=metric&limit={limit}"
    try: 
        response = requests.get(q_url, headers={'token': noaa_key}).json()
        df = pd.DataFrame(response['results'])
        df['station'] = df['station'].str.slice(6,17)
        df = df[['station','value']].rename(columns={'value':d})
        df.to_csv(f'outputs/prcp/{year}/csv/{d}.csv',index=False)
    except:
        dts_chk.append(d)
        print(f'{d} failed')
    time.sleep(1)

### 2.3 Retrieve by Single Date

In [ ]:
# Retrieve data by singular date for dates that failed second attempt to see if there are errors
d='2022-01-01'
q_url = f"{url}datasetid={ds_id}&datatypeid={dt_id}&locationid={loc_id}&startdate={d}&enddate={d}&units=metric&limit={limit}"
response = requests.get(q_url, headers={'token': noaa_key}).json()
df = pd.DataFrame(response['results'])
df.tail(1)

In [ ]:
# Remove prefix from station
df['station'] = df['station'].str.slice(6,17)
df = df[['station','value']].rename(columns={'value':d})
df.tail(1)

In [6]:
# export data to csv
if export: df.to_csv(f'../outputs/ncei-noaa/{year}/csv/{d}.csv',index=False)